# 점별 상호 정보량(PMI)  

단순히 동시에 등장했다고 해서 반드시 연관성이 있는가? 이를 고집한다면 the 같은 녀석들이 유사도가 높다고 나올 것이다. PMI 기준으로 유사도를 재판단해야 한다.

$$ PMI(x, y) = log_2(\frac{P(x, y)}{P(x)P(y)}) $$ 

다만 $$ log_2{0} = -\infin $$ 이므로 max(0)로 한 번 감싼 양의 상호 정보량을 활용한다.

$$ PPMI(x, y) = max(0, PMI(x, y)) $$ 

In [1]:
import sys
sys.path.append("..")
from common.util import ppmi

(array([0, 1, 2, 3, 4]),
 {'wow': 0, 'this': 1, 'is': 2, 'really': 3, 'great': 4},
 {0: 'wow', 1: 'this', 2: 'is', 3: 'really', 4: 'great'})